<a href="https://colab.research.google.com/github/rkmad/contact_parser_mini/blob/main/Web_scraper_Ale_v0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

27 Feb 2022

Web Scraping to obtain CRO and Biotech company contact info

In [ ]:
from bs4 import BeautifulSoup
import requests
from requests.exceptions import HTTPError
from urllib.error import URLError
import pandas as pd
import numpy as np
import lxml

In [ ]:
site_urls = ["https://biopharmguy.com/links/company-by-location-small-molecules.php",
             "https://www.contractresearchmap.com/locations/1/-267.01171874999864/-56.944973484736074/267.0117187499998/76.84081670497531",
             "https://biocomcro.org/cro-directory/"]

# site_urls[0] 1 step scraping: html source file contains all desired information
# site_urls[1] 2 step scraping: html source file has series of url links to secondary pages that contain all desired information
# site_urls[2] 2 step scraping: html source file has series of url links to secondary pages that contain all desired information


# site_urls[0]: BioPharmGuy
# 1 step scraping: html source file contains all desired information

In [ ]:
# get html from first site: BioPharmGuy
site1 = requests.get(site_urls[0])

In [ ]:
# access desired content using beautiful soup
site1_soup = BeautifulSoup(site1.content, "html")
#print(site1_soup.prettify())

In [ ]:
# this is what we want:
# company name
print(site1_soup.find_all('td','company')[0].text)
# url
print(site1_soup.find_all('td','company')[0].a.next_sibling.get('href'))
# description
print(site1_soup.find_all('td','description')[0].text)
# location
print(site1_soup.find_all('td','location')[0].text)
# note: no location provided

Avaliv Therapeutics 
https://www.avaliv.com/
Small molecule therapeutics targeting inflammation and fibrosis related disorders
 - 


In [ ]:
# all four fields present
# company name
print(site1_soup.find_all('td','company')[10].text)
# url
print(site1_soup.find_all('td','company')[10].a.next_sibling.get('href'))
# description
print(site1_soup.find_all('td','description')[10].text)
# location
print(site1_soup.find_all('td','location')[10].text)

Quest PharmaTech 
http://www.questpharmatech.com
Cancer Treatments
AB - Edmonton


In [ ]:
# make lists of fields
td = site1_soup.find_all('td','company')
td_desc = site1_soup.find_all('td','description')
td_loc = site1_soup.find_all('td','location')

In [ ]:
# extract information for all four fields for each company
company_name = []
company_url = []
company_description = []
company_location = []
for idx,item in enumerate(td):
  company_name.append(item.text)
  company_url.append(item.a.next_sibling.get('href'))
  company_description.append(td_desc[idx].text)
  company_location.append(td_loc[idx].text)

In [ ]:
# make dataframe
df1 = pd.DataFrame({'company':company_name, 'website':company_url, 'description':company_description, 'location':company_location})
location_sep = df1['location'].str.split(" - ", n=1, expand=True)
df1['country, US state, or Canadian province'] = location_sep[0]
df1['city'] = location_sep[1]
df1.drop("location", axis=1, inplace=True)

In [ ]:
# BioPharmGuy dataframe
df1

,company,website,description,"country, US state, or Canadian province",city
0,Avaliv Therapeutics,https://www.avaliv.com/,Small molecule therapeutics targeting inflamma...,,
1,Imago Pharmaceuticals,http://www.imago-pharma.com/imago-pharmaceutic...,JNK-directed novel therapies,,
2,Serentrix,https://serentrix.com/,Small Molecule Pain Relief,,
3,Viridia Life Sciences,https://www.viridia.life/,Psychedelics as Therapy,,
4,Xortx Therapeutics,https://www.xortx.com/,Small Molecules,AB,Calgary
...,...,...,...,...,...
2817,Promentis Pharmaceuticals,https://promentispharma.com/,CNS small molecules,WI,Milwaukee
2818,Pfizer,https://www.pfizer.com/,"Testing, Distribution",WI,Pleasant Prairie
2819,Bright Minds Biosciences,https://brightmindsbio.com/,Psychedelics,WI,Wauwatosa
2820,Cmxtwenty,https://cmxtwenty.com/,Small molecule for pain,WI,Wauwatosa


# Run cell below to download Dataframe above containing BioPharmGuy contacts as csv file (only works in Chrome browser)

In [ ]:
df1.to_csv("BioPharmGuy_contacts.csv", index=False)
from google.colab import files
files.download('BioPharmGuy_contacts.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# site_urls[1]: Contract Research Map
# 2 step scraping: html source file has series of url links to secondary pages that contain all desired information

In [ ]:
pages_html = []

for num in range(1,10):
    curr_url = site_urls[1]
    try:
        response =  requests.get(curr_url)
        response.raise_for_status()
        pages_html.append(response)
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
    except Exception as err:
        print(f'Other error occurred: {err}')
    else:
        print(f'Search progress: {num} of {75}', end='\r') # overwrite print on same line


In [ ]:
pages_html

[<Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>,
 <Response [200]>]

In [ ]:
member_dict={}
keys=[]
lastnames=[]
alltext=[]
for response in pages_html:
    xml_tree = BeautifulSoup(response.content, "html")

#     key_temp = xml_tree.find_all('div data-key')
#     print(key_temp)
#     for key in key_temp:
#         keys.append(key.contents[0])

#     lastname_temp = xml_tree.find_all('span',{'style': "text-transform:uppercase;"})#, 'span',{'class': 'surname'}])
#     for lastname in lastname_temp:
#         #[content for content in lastname.contents]
#         lastnames.append([content for content in lastname.contents if str(content) != '<br/>'])


    temp = xml_tree.find_all(['span',{'class': 'name'}])#, 'span',{'class': 'surname'}])
    for temp_text in temp:
        alltext.append([content for content in temp_text.contents if str(content) != '<br/>'])

In [ ]:
# access desired content using beautiful soup
site2_soup = BeautifulSoup(pages_html[0].text, "html")
print(site2_soup.prettify())
#pages_html[0].text

<!DOCTYPE html>
<html>
 <head>
  <title>
   Contract Research Map
  </title>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link href="/assets/application-39f8e70547b82dd32f4189d6316412b8dc6adde93b4eabcfbe99e0778913b65c.css" media="all" rel="stylesheet"/>
  <meta content="authenticity_token" name="csrf-param"/>
  <meta content="tv9WULH6lHBiRTYgpiLaqt5KPh0JQHs5NUW8dBbXgxg58ySZWK6neQFvamerGcnWF9MTYs53taYB9k1O16K9tw==" name="csrf-token"/>
  <!--[if lte IE 8]>
    <link rel="stylesheet" media="screen" href="/stylesheets/mapbox.ie.css" />
  <![endif]-->
  <script crossorigin="anonymous" integrity="sha256-qXBd/EfAdjOA2FGrGAG+b3YBn2tn5A6bhz+LSgYD96k=" src="https://cdn.jsdelivr.net/npm/lodash@4.17.21/lodash.min.js">
  </script>
  <script>
   (function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){
      (i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),
      m=s.getElementsByTagName(o)[0];a.async=1;a.src=g;m.parentNod

site_urls[2]: Contract Research Map
2 step scraping: html source file has series of url links to secondary pages that contain all desired information

In [ ]:
pages_html = []

for num in range(1,10):
    curr_url = site_urls[2]
    try:
        response =  requests.get(curr_url)
        response.raise_for_status()
        pages_html.append(response)
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
    except Exception as err:
        print(f'Other error occurred: {err}')
    else:
        print(f'Search progress: {num} of {75}', end='\r') # overwrite print on same line

In [ ]:
# access desired content using beautiful soup
site3_soup = BeautifulSoup(pages_html[0].text, "html")
print(site3_soup.prettify())
#pages_html[0].text

<!DOCTYPE html>
<html lang="en-US">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link href="https://gmpg.org/xfn/11" rel="profile"/>
  <link href="https://biocomcro.org/xmlrpc.php" rel="pingback"/>
  <script src="https://use.typekit.net/wlp5edg.js">
  </script>
  <script>
   try{Typekit.load({ async: true });}catch(e){}
  </script>
  <link href="https://biocomcro.org/wp-content/themes/biocom-cro/images/favicons/apple-touch-icon-57x57.png" rel="apple-touch-icon" sizes="57x57"/>
  <link href="https://biocomcro.org/wp-content/themes/biocom-cro/images/favicons/apple-touch-icon-60x60.png" rel="apple-touch-icon" sizes="60x60"/>
  <link href="https://biocomcro.org/wp-content/themes/biocom-cro/images/favicons/apple-touch-icon-72x72.png" rel="apple-touch-icon" sizes="72x72"/>
  <link href="https://biocomcro.org/wp-content/themes/biocom-cro/images/favicons/apple-touch-icon-76x76.png" rel="apple-touch-icon" sizes="76x76"/>
  <link hr

In [ ]:
url4 = 'https://biocomcro.org/directory/drug-discovery/'
url4_html = requests.get(url4)

# access desired content using beautiful soup
url4_soup = BeautifulSoup(url4_html.text, "html")
print(url4_soup.prettify())

<!DOCTYPE html>
<html lang="en-US">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link href="https://gmpg.org/xfn/11" rel="profile"/>
  <link href="https://biocomcro.org/xmlrpc.php" rel="pingback"/>
  <script src="https://use.typekit.net/wlp5edg.js">
  </script>
  <script>
   try{Typekit.load({ async: true });}catch(e){}
  </script>
  <link href="https://biocomcro.org/wp-content/themes/biocom-cro/images/favicons/apple-touch-icon-57x57.png" rel="apple-touch-icon" sizes="57x57"/>
  <link href="https://biocomcro.org/wp-content/themes/biocom-cro/images/favicons/apple-touch-icon-60x60.png" rel="apple-touch-icon" sizes="60x60"/>
  <link href="https://biocomcro.org/wp-content/themes/biocom-cro/images/favicons/apple-touch-icon-72x72.png" rel="apple-touch-icon" sizes="72x72"/>
  <link href="https://biocomcro.org/wp-content/themes/biocom-cro/images/favicons/apple-touch-icon-76x76.png" rel="apple-touch-icon" sizes="76x76"/>
  <link hr